In [1]:
%load_ext autoreload
%load_ext line_profiler

In [2]:
from linear_models import LinearOneVsAllClassifier
import numpy as np
import noise_functions_multi as noise
import time
import ray
from utils import find_noise_bounds_multi
%autoreload 2

In [3]:
ray.init()

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:64283 to respond...
Waiting for redis server at 127.0.0.1:56005 to respond...
Starting local scheduler with the following resources: {'GPU': 0, 'CPU': 4}.

View the web UI at http://localhost:8901/notebooks/ray_ui28413.ipynb?token=516c85f82bbc49e0fbd22631acd137060e222be972b8cec5



{'local_scheduler_socket_names': ['/tmp/scheduler14902983'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store30836672', manager_name='/tmp/plasma_manager19936828', manager_port=44441)],
 'redis_address': '127.0.0.1:64283',
 'webui_url': 'http://localhost:8901/notebooks/ray_ui28413.ipynb?token=516c85f82bbc49e0fbd22631acd137060e222be972b8cec5'}

In [4]:
linear_models = [LinearOneVsAllClassifier(10, np.load('linear_models/multi/w_{}.npy'.format(i)), np.load('linear_models/multi/b_{}.npy'.format(i)))
                 for i in range(5)]

In [5]:
X_exp = np.load('linear_experiments_data/multi/X_exp.npy')
Y_exp = np.load('linear_experiments_data/multi/Y_exp.npy')

In [22]:
linear_models[0].predict(X_exp[:1])

array([7])

In [44]:
%lprun -f noise.grad_desc_convex noise.grad_desc_convex([1], linear_models[:1], X_exp[0], Y_exp[0], .5)

Timer unit: 1e-06 s

Total time: 1.03265 s
File: /Users/juanperdomo/universal_noise_generators/noise_functions_multi.py
Function: grad_desc_convex at line 112

Line #      Hits         Time  Per Hit   % Time  Line Contents
   112                                           def grad_desc_convex(distribution, models, x, y, alpha, target=False, learning_rate=.01, iters=2000, early_stop=5,
   113                                                                box_min=0.0, box_max=1.0):
   114         1         11.0     11.0      0.0      if target:
   115                                                   return grad_desc_targeted(distribution, models, x, target, alpha, learning_rate, iters, early_stop, box_min,
   116                                                                             box_max)[1]
   117                                               else:
   118         1          4.0      4.0      0.0          other_labels = range(10)
   119         1          2.0      2.0      0.0   

In [43]:
print np.linalg.norm(v)
linear_models[0].predict((X_exp[0] + v).reshape(1,-1))

0.5953985827438367


array([3])

In [ ]:
find_noise_bounds_multi(linear_models[:1], X_exp[:1])

In [ ]:
np.linalg.norm(v[1])

In [ ]:
s = time.time()
results = [noise.grad_desc_nonconvex.remote([1,1,1,1,1], linear_models, X_exp[i], Y_exp[i], 1) for i in xrange(500)]
res = ray.get(results)
print time.time() - s 

In [ ]:
s = time.time()
results = [noise.grad_desc_nonconvex([1,1,1,1,1], linear_models, X_exp[i], Y_exp[i], 1) for i in xrange(500)]
print time.time() - s 

In [ ]:
[model.evaluate((X_exp[0] + v).reshape(1,-1), Y_exp[0]) for model in linear_models]